# 0. 공통 import & 경로

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import ast
import json
import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

from sklearn.model_selection import StratifiedKFold, train_test_split

In [3]:
import os

BASE_DIR = "/content/drive/MyDrive/cv-medislr/data"
PRE_DIR  = os.path.join(BASE_DIR, "preprocessed")

HAND_IMG_ROOT = os.path.join(PRE_DIR, "holistic_frames", "hands")
TSN_META_PATH = os.path.join(PRE_DIR, "metadata", "tsn_hands_seq_meta.csv")

print("HAND_IMG_ROOT:", HAND_IMG_ROOT)
print("TSN_META_PATH:", TSN_META_PATH)
print("META exists:", os.path.exists(TSN_META_PATH))

HAND_IMG_ROOT: /content/drive/MyDrive/cv-medislr/data/preprocessed/holistic_frames/hands
TSN_META_PATH: /content/drive/MyDrive/cv-medislr/data/preprocessed/metadata/tsn_hands_seq_meta.csv
META exists: True


In [4]:
# (추가) frames_img 안에 박혀있는 "옛 경로"를 "새 경로"로 바꿔줄 루트
OLD_HAND_ROOT = "/content/drive/MyDrive/preprocessing/Holistic_hands_frames"
NEW_HAND_ROOT = "/content/drive/MyDrive/cv-medislr/data/preprocessed/holistic_frames/hands"

# 1. TSN 메타 로드 + frames_img를 리스트로 복원

In [5]:
tsn_df = pd.read_csv(TSN_META_PATH)
print(tsn_df.head())
print(tsn_df.columns)
print("rows:", len(tsn_df))

                                       seq_id  person_id word_code  label_idx  \
0  p1_WORD0029_검사_NIA_SL_WORD0029_REAL01_D          1  WORD0029          0   
1  p1_WORD0029_검사_NIA_SL_WORD0029_REAL01_F          1  WORD0029          0   
2  p1_WORD0029_검사_NIA_SL_WORD0029_REAL01_L          1  WORD0029          0   
3  p1_WORD0029_검사_NIA_SL_WORD0029_REAL01_R          1  WORD0029          0   
4  p1_WORD0029_검사_NIA_SL_WORD0029_REAL01_U          1  WORD0029          0   

  view                                         frames_img  
0    D  ['/content/drive/MyDrive/preprocessing/Holisti...  
1    F  ['/content/drive/MyDrive/preprocessing/Holisti...  
2    L  ['/content/drive/MyDrive/preprocessing/Holisti...  
3    R  ['/content/drive/MyDrive/preprocessing/Holisti...  
4    U  ['/content/drive/MyDrive/preprocessing/Holisti...  
Index(['seq_id', 'person_id', 'word_code', 'label_idx', 'view', 'frames_img'], dtype='object')
rows: 1097


In [6]:
def parse_frames_cell(val):
    # 이미 리스트이면 그대로
    if isinstance(val, list):
        return val
    # 문자열이면 리스트 리터럴일 가능성
    if isinstance(val, str):
        s = val.strip()
        if not s:
            return []
        try:
            return ast.literal_eval(s)
        except Exception:
            # 그냥 하나의 경로만 문자열인 경우
            return [s]
    return []

# (추가) 옛 경로 -> 새 경로 치환
def remap_frame_path(p: str) -> str:
    p = str(p)
    if p.startswith(OLD_HAND_ROOT):
        p = p.replace(OLD_HAND_ROOT, NEW_HAND_ROOT, 1)
    return p

tsn_df["frames_img"] = tsn_df["frames_img"].apply(parse_frames_cell)

# (추가) frames_img 내부 경로를 새 루트로 remap
tsn_df["frames_img"] = tsn_df["frames_img"].apply(lambda lst: [remap_frame_path(x) for x in lst])

import os
sample_path = tsn_df.iloc[0]["frames_img"][0]
print("sample_path:", sample_path)
print("exists:", os.path.exists(sample_path))

print(type(tsn_df.iloc[0]["frames_img"]), len(tsn_df.iloc[0]["frames_img"]))
print(tsn_df.iloc[0]["frames_img"][:3])

sample_path: /content/drive/MyDrive/cv-medislr/data/preprocessed/holistic_frames/hands/1/WORD0029_검사_NIA_SL_WORD0029_REAL01_D_s00.png
exists: True
<class 'list'> 16
['/content/drive/MyDrive/cv-medislr/data/preprocessed/holistic_frames/hands/1/WORD0029_검사_NIA_SL_WORD0029_REAL01_D_s00.png', '/content/drive/MyDrive/cv-medislr/data/preprocessed/holistic_frames/hands/1/WORD0029_검사_NIA_SL_WORD0029_REAL01_D_s01.png', '/content/drive/MyDrive/cv-medislr/data/preprocessed/holistic_frames/hands/1/WORD0029_검사_NIA_SL_WORD0029_REAL01_D_s02.png']


# 2. Grayscale 변환 + Transform 정의
손 skeleton은 흰 선만 있으니 1채널(grayscale) 이 효율적.

In [7]:
IMG_SIZE = 64

train_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),                       # (1, H, W)
])

eval_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
])

# 3. 손 부분 crop 함수 + Dataset 정의

In [8]:
def crop_to_hand(img_pil, margin=50):
    """
    img_pil: PIL.Image (grayscale)
    0이 아닌 픽셀의 bounding box를 찾아 crop
    """
    arr = np.array(img_pil)

    # grayscale 변환
    if arr.ndim == 3:
        gray = arr.mean(axis=2)
    else:
        gray = arr

    # non-zero 픽셀 찾기
    ys, xs = np.nonzero(gray)

    # 만약 skeleton이 1픽셀도 없으면 원본 그대로 반환
    if len(xs) == 0 or len(ys) == 0:
        return img_pil

    # bounding box
    x_min, x_max = xs.min(), xs.max()
    y_min, y_max = ys.min(), ys.max()

    # margin 추가
    x_min = max(0, x_min - margin)
    y_min = max(0, y_min - margin)
    x_max = min(gray.shape[1] - 1, x_max + margin)
    y_max = min(gray.shape[0] - 1, y_max + margin)

    # crop
    return img_pil.crop((x_min, y_min, x_max + 1, y_max + 1))

In [9]:
# # 3-2. HandTSNDataset

# FRAMES_COL = "frames_img"

# class HandTSNDataset(Dataset):
#     def __init__(self, df, transform=None):
#         self.df = df.reset_index(drop=True)
#         self.transform = transform

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
#         frame_paths = row[FRAMES_COL]
#         label = int(row["label_idx"])

#         frames = []
#         for p in frame_paths:
#             img = Image.open(p).convert("L")      # 1채널로 읽기
#             img = crop_to_hand(img, margin=50)   # ★ skeleton 주변만 crop

#             if self.transform is not None:
#                 img = self.transform(img)        # (1, H, W)

#             frames.append(img)

#         frames = torch.stack(frames, dim=0)      # (T, C, H, W)

#         return frames, torch.tensor(label, dtype=torch.long)

# (추가) 전처리용 transform 정의

In [10]:
import torchvision.transforms as T

IMG_SIZE = 64  # 이미 이 크기로 쓰고 있으면 그대로

preprocess_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),                       # (1, H, W)
])

In [11]:
import torch
import os
from tqdm import tqdm

# 텐서 저장 폴더 (원하는 위치로 바꿔도 됨)
TENSOR_ROOT = os.path.join(BASE_DIR, "Hand_tensors")
os.makedirs(TENSOR_ROOT, exist_ok=True)

tensor_paths = []  # 각 시퀀스의 .pt 경로를 저장할 리스트

for idx, row in tqdm(tsn_df.iterrows(), total=len(tsn_df)):
    frame_paths = row["frames_img"]   # 길이 16 리스트
    seq_tensors = []

    for p in frame_paths:
        img = Image.open(p).convert("L")        # 그레이스케일
        img = crop_to_hand(img, margin=50)      # 손 주변만 crop
        img = preprocess_transform(img)         # (1, H, W), float32
        seq_tensors.append(img)

    seq_tensor = torch.stack(seq_tensors, dim=0)   # (T, C, H, W) = (16, 1, H, W)

    # 파일명: seq_{원래 row index}.pt (원하면 seq_id 써도 됨)
    tensor_name = f"seq_{idx:05d}.pt"
    tensor_path = os.path.join(TENSOR_ROOT, tensor_name)

    torch.save(seq_tensor, tensor_path)
    tensor_paths.append(tensor_path)

# tsn_df에 새 컬럼 추가
tsn_df["tensor_path"] = tensor_paths

print(tsn_df[["tensor_path", "label_idx"]].head())

# 메타 저장(선택)
TENSOR_META_PATH = os.path.join(TENSOR_ROOT, "tsn_hands_tensor_meta.csv")
tsn_df.to_csv(TENSOR_META_PATH, index=False)
print("텐서 메타 저장:", TENSOR_META_PATH)

100%|██████████| 1097/1097 [1:48:39<00:00,  5.94s/it]

                                         tensor_path  label_idx
0  /content/drive/MyDrive/cv-medislr/data/Hand_te...          0
1  /content/drive/MyDrive/cv-medislr/data/Hand_te...          0
2  /content/drive/MyDrive/cv-medislr/data/Hand_te...          0
3  /content/drive/MyDrive/cv-medislr/data/Hand_te...          0
4  /content/drive/MyDrive/cv-medislr/data/Hand_te...          0
텐서 메타 저장: /content/drive/MyDrive/cv-medislr/data/Hand_tensors/tsn_hands_tensor_meta.csv


In [14]:
import os
import pandas as pd

NEW_ROOT = "/content/drive/MyDrive/cv-medislr/data/preprocessed/tensors/tsn_hands"
META_PATH = f"{NEW_ROOT}/tsn_hands_tensor_meta.csv"

print("META_PATH:", META_PATH)
print("exists:", os.path.exists(META_PATH))

df = pd.read_csv(META_PATH)

def remap_tensor_path(p):
    fname = os.path.basename(p)   # seq_00001.pt
    return os.path.join(NEW_ROOT, fname)

df["tensor_path"] = df["tensor_path"].apply(remap_tensor_path)

df.to_csv(META_PATH, index=False)
print("tensor_path remap 완료")

META_PATH: /content/drive/MyDrive/cv-medislr/data/preprocessed/tensors/tsn_hands/tsn_hands_tensor_meta.csv
exists: True
tensor_path remap 완료


In [15]:
# 텐서 메타 다시 로드 (런타임 새로 켰을 때용)

import pandas as pd
import torch
import os
import numpy as np

BASE_DIR = "/content/drive/MyDrive/cv-medislr/data"
PRE_DIR  = os.path.join(BASE_DIR, "preprocessed")

# ★ 여기 중요
TENSOR_ROOT = os.path.join(PRE_DIR, "tensors", "tsn_hands")

TENSOR_META_PATH = os.path.join(
    TENSOR_ROOT,
    "tsn_hands_tensor_meta.csv"
)

print("TENSOR_META_PATH:", TENSOR_META_PATH)
print("exists:", os.path.exists(TENSOR_META_PATH))

tensor_df = pd.read_csv(TENSOR_META_PATH)
print(tensor_df.columns)
print(tensor_df[["tensor_path", "label_idx"]].head())
print("rows:", len(tensor_df))

# tsn_df 대신 이제 tensor_df를 쓰게 될 거야

TENSOR_META_PATH: /content/drive/MyDrive/cv-medislr/data/preprocessed/tensors/tsn_hands/tsn_hands_tensor_meta.csv
exists: True
Index(['seq_id', 'person_id', 'word_code', 'label_idx', 'view', 'frames_img',
       'tensor_path'],
      dtype='object')
                                         tensor_path  label_idx
0  /content/drive/MyDrive/cv-medislr/data/preproc...          0
1  /content/drive/MyDrive/cv-medislr/data/preproc...          0
2  /content/drive/MyDrive/cv-medislr/data/preproc...          0
3  /content/drive/MyDrive/cv-medislr/data/preproc...          0
4  /content/drive/MyDrive/cv-medislr/data/preproc...          0
rows: 1097


# 5. MobileNetTSN (경량 TSN 모델(Small CNN)에서 교체함)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [17]:
# class SmallHandTSN(nn.Module):
#     def __init__(self, num_classes=22):
#         super().__init__()
#         self.features = nn.Sequential(
#             nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),  # (B*T,32,H/2,W/2)
#             nn.BatchNorm2d(32),
#             nn.ReLU(inplace=True),

#             nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1), # (B*T,64,H/4,W/4)
#             nn.BatchNorm2d(64),
#             nn.ReLU(inplace=True),

#             nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),# (B*T,128,H/8,W/8)
#             nn.BatchNorm2d(128),
#             nn.ReLU(inplace=True),
#         )
#         self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
#         self.dropout = nn.Dropout(p=0.5)
#         self.fc = nn.Linear(128, num_classes)

#     def forward(self, x):
#         """
#         x: (B, T, C, H, W)  여기서 C=1
#         """
#         B, T, C, H, W = x.shape
#         x = x.view(B * T, C, H, W)     # (B*T, C, H, W)
#         feat = self.features(x)        # (B*T, 128, h, w)
#         feat = self.global_pool(feat)  # (B*T, 128, 1, 1)
#         feat = feat.view(B, T, 128)    # (B, T, 128)
#         feat = feat.mean(dim=1)        # TSN: 시간 평균 (B, 128)
#         feat = self.dropout(feat)
#         logits = self.fc(feat)         # (B, num_classes)
#         return logits

In [18]:
import torch.nn as nn
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

class MobileNetTSN(nn.Module):
    def __init__(self, num_classes=22, pretrained=True):
        super().__init__()
        # ImageNet pretrained MobileNetV2 불러오기
        weights = MobileNet_V2_Weights.IMAGENET1K_V1 if pretrained else None
        backbone = mobilenet_v2(weights=weights)

        self.features = backbone.features           # conv 부분
        self.last_channel = backbone.last_channel   # 1280
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(self.last_channel, num_classes)

        # ImageNet 정규화용 mean/std (0~1 입력 기준)
        self.register_buffer(
            "img_mean",
            torch.tensor([0.485, 0.456, 0.406]).view(1, 1, 3, 1, 1)
        )
        self.register_buffer(
            "img_std",
            torch.tensor([0.229, 0.224, 0.225]).view(1, 1, 3, 1, 1)
        )

    def forward(self, x):
        """
        x: (B, T, C, H, W)  여기서 C=1 (grayscale)
        """
        B, T, C, H, W = x.shape

        # 1채널 → 3채널 replicate (pretrained weight 활용 위해)
        if C == 1:
            x = x.repeat(1, 1, 3, 1, 1)   # (B, T, 3, H, W)

        # ImageNet 정규화
        x = (x - self.img_mean) / self.img_std

        # (B*T, 3, H, W)로 reshape
        x = x.view(B * T, 3, H, W)

        feat = self.features(x)          # (B*T, C2, h, w)
        feat = self.pool(feat)           # (B*T, C2, 1, 1)
        feat = feat.view(B, T, self.last_channel)  # (B, T, C2)

        # TSN: 시간 평균 풀링
        feat = feat.mean(dim=1)          # (B, C2)

        feat = self.dropout(feat)
        logits = self.fc(feat)           # (B, num_classes)
        return logits

# 6. 학습/검증/테스트 루프

In [19]:
import torch
import torch.nn as nn

def run_epoch(model, loader, criterion, optimizer=None, log_interval=None):
    """
    optimizer: None이면 eval, 아니면 train
    log_interval: 정수면 batch_log 출력, None이면 출력 안 함
    """
    if optimizer is None:
        model.eval()
        torch.set_grad_enabled(False)
    else:
        model.train()
        torch.set_grad_enabled(True)

    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    for batch_idx, (frames, labels) in enumerate(loader):
        # 너무 자주 안 찍히지 않게 interval로 제어
        if (log_interval is not None) and (batch_idx % log_interval == 0):
            print(f"    batch {batch_idx}/{len(loader)} ...")

        frames = frames.to(device)   # (B, T, C, H, W)
        labels = labels.to(device)   # (B,)

        if optimizer is not None:
            optimizer.zero_grad()

        logits = model(frames)       # (B, num_classes)
        loss = criterion(logits, labels)

        if optimizer is not None:
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=3.0)
            optimizer.step()

        bs = labels.size(0)
        total_loss += loss.item() * bs
        preds = logits.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += bs

    avg_loss = total_loss / total_samples
    avg_acc  = total_correct / total_samples

    return avg_loss, avg_acc

In [20]:
# 인덱스 70/15/15로 나누기

from sklearn.model_selection import train_test_split

TRAIN_RATIO = 0.7
VAL_RATIO   = 0.15
TEST_RATIO  = 0.15
assert abs(TRAIN_RATIO + VAL_RATIO + TEST_RATIO - 1.0) < 1e-6

indices = np.arange(len(tensor_df))
labels  = tensor_df["label_idx"].values  # 0~21

# 1) train vs temp
train_idx, temp_idx, y_train, y_temp = train_test_split(
    indices,
    labels,
    test_size=VAL_RATIO + TEST_RATIO,
    random_state=42,
    stratify=labels,
)

# 2) temp를 val/test로
val_ratio_rel = VAL_RATIO / (VAL_RATIO + TEST_RATIO)

val_idx, test_idx = train_test_split(
    temp_idx,
    test_size=1.0 - val_ratio_rel,
    random_state=42,
    stratify=y_temp,
)

print("train:", len(train_idx), "val:", len(val_idx), "test:", len(test_idx))

# ===== (추가) split 인덱스 저장 =====
SPLIT_PATH = os.path.join(TENSOR_ROOT, "split_70_15_15.json")
split_dict = {
    "train_idx": train_idx.tolist(),
    "val_idx": val_idx.tolist(),
    "test_idx": test_idx.tolist(),
    "seed": 42
}
with open(SPLIT_PATH, "w") as f:
    json.dump(split_dict, f, indent=2)
print("Saved split:", SPLIT_PATH)
# ====================================

train_df = tensor_df.iloc[train_idx].reset_index(drop=True)
val_df   = tensor_df.iloc[val_idx].reset_index(drop=True)
test_df  = tensor_df.iloc[test_idx].reset_index(drop=True)

print("train_df:", train_df.shape)
print("val_df:", val_df.shape)
print("test_df:", test_df.shape)

train: 767 val: 165 test: 165
Saved split: /content/drive/MyDrive/cv-medislr/data/preprocessed/tensors/tsn_hands/split_70_15_15.json
train_df: (767, 7)
val_df: (165, 7)
test_df: (165, 7)


In [21]:
# .pt 텐서용 Dataset 정의

from torch.utils.data import Dataset

class TensorSeqDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        tensor_path = row["tensor_path"]
        label = int(row["label_idx"])

        seq_tensor = torch.load(tensor_path)          # (T, C, H, W)
        seq_tensor = seq_tensor.float()               # 안전하게 float32 보장

        return seq_tensor, torch.tensor(label, dtype=torch.long)

In [22]:
# 새 DataLoader 헬퍼 (get_loaders)

from torch.utils.data import DataLoader

def get_loaders(batch_size=8, num_workers=1):
    train_ds = TensorSeqDataset(train_df)
    val_ds   = TensorSeqDataset(val_df)
    test_ds  = TensorSeqDataset(test_df)

    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=True
    )
    val_loader = DataLoader(
        val_ds, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=True
    )
    test_loader = DataLoader(
        test_ds, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=True
    )
    return train_loader, val_loader, test_loader

In [23]:
from copy import deepcopy
import os

def train_model(num_epochs=8, lr=1e-3, weight_decay=1e-4,
                batch_size=8, log_interval=20):
    print("\n===== Training 시작 =====")

    train_loader, val_loader, test_loader = get_loaders(
        batch_size=batch_size,
        num_workers=1,
    )

    model = MobileNetTSN(num_classes=22, pretrained=True).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="max", factor=0.5, patience=3
    )

    best_val_acc = 0.0
    best_state = None
    patience = 5
    no_improve = 0

    for epoch in range(1, num_epochs + 1):
        train_loss, train_acc = run_epoch(
            model, train_loader, criterion,
            optimizer=optimizer,
            log_interval=log_interval,
        )
        val_loss, val_acc = run_epoch(
            model, val_loader, criterion,
            optimizer=None,
            log_interval=None,
        )

        scheduler.step(val_acc)

        print(f"[Epoch {epoch:02d}] "
              f"train_loss={train_loss:.4f}, train_acc={train_acc:.4f} | "
              f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = deepcopy(model.state_dict())
            no_improve = 0
        else:
            no_improve += 1

        if no_improve >= patience:
            print(f"Early stopping at epoch {epoch} (no_improve={no_improve})")
            break

    # best 모델로 test 평가
    model.load_state_dict(best_state)
    test_loss, test_acc = run_epoch(
        model, test_loader, criterion,
        optimizer=None,
        log_interval=None,
    )
    print(f"[Final] best_val_acc={best_val_acc:.4f}, test_acc={test_acc:.4f}")

    # ===== best checkpoint 저장 (경로 수정됨) =====
    BASE_DIR = "/content/drive/MyDrive/cv-medislr/data"
    PRE_DIR  = os.path.join(BASE_DIR, "preprocessed")

    CKPT_DIR = os.path.join(PRE_DIR, "model_weights", "2d_only")
    os.makedirs(CKPT_DIR, exist_ok=True)

    CKPT_PATH = os.path.join(CKPT_DIR, "mobilenet_tsn_hands_best.pt")
    torch.save({
        "model_name": "MobileNetTSN",
        "model_group": "2d_only",
        "num_classes": 22,
        "img_size": IMG_SIZE,
        "state_dict": best_state,
        "best_val_acc": float(best_val_acc),
        "test_acc": float(test_acc),
    }, CKPT_PATH)

    print("Saved checkpoint:", CKPT_PATH)
    # ============================================

    return {
        "best_val_acc": float(best_val_acc),
        "test_acc": float(test_acc),
        "ckpt_path": CKPT_PATH,
    }

In [24]:
# 최종 실행

results = []

stats = train_model(
    num_epochs=15,      # 먼저 8 정도로 테스트
    lr=1e-3,
    weight_decay=1e-4,
    batch_size=8,
    log_interval=20,  # 20 스텝마다 batch 로그; 귀찮으면 None
)
results.append(stats)

results_df = pd.DataFrame(results)
print("\n===== 최종 결과 =====")
print(results_df)


===== Training 시작 =====
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 113MB/s]


    batch 0/96 ...
    batch 20/96 ...
    batch 40/96 ...
    batch 60/96 ...
    batch 80/96 ...
[Epoch 01] train_loss=2.5971, train_acc=0.2451 | val_loss=2.9760, val_acc=0.2848
    batch 0/96 ...
    batch 20/96 ...
    batch 40/96 ...
    batch 60/96 ...
    batch 80/96 ...
[Epoch 02] train_loss=1.6956, train_acc=0.4628 | val_loss=1.4556, val_acc=0.5515
    batch 0/96 ...
    batch 20/96 ...
    batch 40/96 ...
    batch 60/96 ...
    batch 80/96 ...
[Epoch 03] train_loss=1.5469, train_acc=0.5463 | val_loss=1.1839, val_acc=0.6121
    batch 0/96 ...
    batch 20/96 ...
    batch 40/96 ...
    batch 60/96 ...
    batch 80/96 ...
[Epoch 04] train_loss=1.2923, train_acc=0.5893 | val_loss=1.3559, val_acc=0.5212
    batch 0/96 ...
    batch 20/96 ...
    batch 40/96 ...
    batch 60/96 ...
    batch 80/96 ...
[Epoch 05] train_loss=1.1751, train_acc=0.6297 | val_loss=1.5913, val_acc=0.5636
    batch 0/96 ...
    batch 20/96 ...
    batch 40/96 ...
    batch 60/96 ...
    batch 80/96 ...
[

# Demo 용 20개 샘플 추출

In [25]:
import os

path = "/content/drive/MyDrive/cv-medislr/data/preprocessed/tensors/tsn_hands"
print("exists:", os.path.exists(path))

if os.path.exists(path):
    print("files:", os.listdir(path)[:10])

exists: True
files: ['seq_00000.pt', 'seq_00001.pt', 'seq_00002.pt', 'seq_00003.pt', 'seq_00004.pt', 'seq_00005.pt', 'seq_00006.pt', 'seq_00007.pt', 'seq_00008.pt', 'seq_00009.pt']


In [26]:
import pandas as pd
import os

BASE_DIR = "/content/drive/MyDrive/cv-medislr/data"
TENSOR_ROOT = os.path.join(BASE_DIR, "preprocessed/tensors/tsn_hands")

META_PATH = os.path.join(TENSOR_ROOT, "tsn_hands_tensor_meta.csv")

print("META exists:", os.path.exists(META_PATH))

tensor_df = pd.read_csv(META_PATH)
print(tensor_df.columns)
print(tensor_df[["tensor_path", "label_idx"]].head())
print("rows:", len(tensor_df))

META exists: True
Index(['seq_id', 'person_id', 'word_code', 'label_idx', 'view', 'frames_img',
       'tensor_path'],
      dtype='object')
                                         tensor_path  label_idx
0  /content/drive/MyDrive/cv-medislr/data/preproc...          0
1  /content/drive/MyDrive/cv-medislr/data/preproc...          0
2  /content/drive/MyDrive/cv-medislr/data/preproc...          0
3  /content/drive/MyDrive/cv-medislr/data/preproc...          0
4  /content/drive/MyDrive/cv-medislr/data/preproc...          0
rows: 1097


In [27]:
import os
import shutil
import pandas as pd
import random

# ===== 경로 설정 =====
BASE_DIR = "/content/drive/MyDrive/cv-medislr/data"

SRC_TENSOR_DIR = os.path.join(
    BASE_DIR, "preprocessed/tensors/tsn_hands"
)
META_PATH = os.path.join(
    SRC_TENSOR_DIR, "tsn_hands_tensor_meta.csv"
)

DST_SAMPLE_DIR = os.path.join(
    BASE_DIR, "samples/2d_only/skeleton_tsn"
)
os.makedirs(DST_SAMPLE_DIR, exist_ok=True)

# ===== 메타 로드 =====
df = pd.read_csv(META_PATH)
print("total tensors:", len(df))

# ===== 20개 랜덤 샘플 =====
N_SAMPLES = 20
sample_df = df.sample(n=N_SAMPLES, random_state=42).reset_index(drop=True)

# ===== pt 파일 복사 =====
new_tensor_paths = []

for i, row in sample_df.iterrows():
    src_path = row["tensor_path"]
    fname = os.path.basename(src_path)

    dst_path = os.path.join(DST_SAMPLE_DIR, fname)
    shutil.copy2(src_path, dst_path)

    new_tensor_paths.append(dst_path)

# ===== 샘플 메타 저장 =====
sample_df["tensor_path"] = new_tensor_paths
SAMPLE_META_PATH = os.path.join(DST_SAMPLE_DIR, "tsn_sample_meta.csv")
sample_df.to_csv(SAMPLE_META_PATH, index=False)

print(" 샘플 저장 완료")
print("sample dir:", DST_SAMPLE_DIR)
print("meta:", SAMPLE_META_PATH)

total tensors: 1097
 샘플 저장 완료
sample dir: /content/drive/MyDrive/cv-medislr/data/samples/2d_only/skeleton_tsn
meta: /content/drive/MyDrive/cv-medislr/data/samples/2d_only/skeleton_tsn/tsn_sample_meta.csv
